# Context Managers

## Context Manager in Python

Python came out with a special new keyword several years ago in Python 2.5 that is known as the with statement. This new keyword allows a developer to create context managers. But wait! What’s a context manager? They are handy constructs that allow you to set something up and tear something down automatically. For example, you might want to open a file, write a bunch of stuff to it and then close it. 

In [1]:
path = './3_context_manager_demos/context_manager_data.txt'

In [2]:
# Using the "with" special keyword to build a context manager

with open(path, 'w') as f_obj:
    f_obj.write('Jaque')

In [3]:
# This is how you had to do it before python 2.5

f_obj = open(path, 'w')
f_obj.write('Jaqueline')
f_obj.close()

## Creating a Context Manager class

In [12]:
import sqlite3


class DataConn:
    """"""

    def __init__(self, db_name):
        """Constructor"""
        self.db_name = db_name

    def __enter__(self):
        """
        Open the database connection
        """
        self.conn = sqlite3.connect(self.db_name)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Close the connection
        """
        self.conn.close()
        if exc_val:
            raise

if __name__ == '__main__':
    db = './3_context_manager_demos/test.db'
    # __enter__ is ran behind the scenes here
    with DataConn(db) as conn:
        cursor = conn.cursor()
    # __exit__ is ran behind the scenes here

## Creating a Context Manager using contextlib

In [13]:
# We can use the contextlib contextmanager decorator to allow us to use a function as a context manager

from contextlib import contextmanager

@contextmanager
def file_open(path):
    try:
        f_obj = open(path, 'w')
        yield f_obj
    except OSError:
        print("We had an error!")
    finally:
        print('Closing file')
        f_obj.close()

if __name__ == '__main__':
    with file_open(path) as fobj:
        fobj.write('Testing context managers')

Closing file


## contextlib.closing(thing)

In [14]:
# Using the closing class instead of the contextmanager decorator

from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('http://www.google.com')) as webpage:
    for line in webpage:
        # process the line
        pass

## contextlib.suppress(*exceptions)

Another handy little tool is the suppress class which was added in Python 3.4. The idea behind this context manager utility is that it can suppress any number of exceptions. Let’s say we want to ignore the FileNotFoundError exception. 

In [15]:
with open('fauxfile.txt') as fobj:
    for line in fobj:
        print(line)

#Traceback (most recent call last):
#   File "/usercode/__ed_file.py", line 1, in <module>
# with open('fauxfile.txt') as fobj:
#FileNotFoundError: [Errno 2] No such file or directory: 'fauxfile.txt''

FileNotFoundError: [Errno 2] No such file or directory: 'fauxfile.txt'

In [16]:
from contextlib import suppress

with suppress(FileNotFoundError):
    with open('fauxfile.txt') as fobj:
        for line in fobj:
            print(line)

## contextlib.redirect_stdout/redirect_stderr

In [ ]:
# Redirecting outputs with vanilla python

import sys

with open(path, 'w') as fobj:
    sys.stdout = fobj
    help(sum)

In [ ]:
# Redirecting output using the contextlib library function redirect_stdout

from contextlib import redirect_stdout

with open(path, 'w') as fobj:
    with redirect_stdout(fobj):
        help(redirect_stdout)

## ExistStack

ExitStack is a context manager that will allow you to easily programmatically combine other context managers and cleanup functions.

In [ ]:
# This code basically creates a series of context managers inside the list comprehension. 
# The ExitStack maintains a stack of registered callbacks that it will call in reverse order
# when the instance it closed, which happens when we exit the the bottom of the with statement.

filenames = [path]

from contextlib import ExitStack
with ExitStack() as stack:
    file_objects = [stack.enter_context(open(filename))
        for filename in filenames]

## Reentrant Context Managers

In [ ]:
# Most context managers are written to ONLY be used in a with statement

from contextlib import contextmanager
@contextmanager
def single():
    print('Yielding')
    yield
    print('Exiting context manager')
    
# Example of running
context = single()
with context:
    pass

# Example of error
with context:
    pass

In [ ]:
# Creating a reentrant context manager (i.e. can be used more than once)

from contextlib import redirect_stdout
from io import StringIO

stream = StringIO()
write_to_stream = redirect_stdout(stream)

with write_to_stream:
    print('Write something to the stream')

with write_to_stream:
    print('Write something else to stream')

print(stream.getvalue())
#Write something to the stream
#Write something else to stream